# **Installing the dependencies**

In [1]:
!pip install transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.5 MB/s eta 0:00:00


In [2]:
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.7 MB/s eta 0:00:00


In [3]:
!pip install -U SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.0 MB/s eta 0:00:00


In [4]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.4 MB/s eta 0:00:00


In [5]:
CUDA_LAUNCH_BLOCKING=1 # to stop cuda blocking

# **Connecting google colab to drive**

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Importing the dependencies**

In [7]:
import json
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification, T5Tokenizer, T5ForSequenceClassification,TrainingArguments, Trainer

# **Loading the data and solution of the surprise dataset**

In [8]:
import json
import pandas as pd
data_path="/content/drive/MyDrive/intent_classification_It_bombay/surprise_data/surprise.data"
data_label_path="/content/drive/MyDrive/intent_classification_It_bombay/surprise_data/surprise.solution"

In [9]:
with open(data_path,'r') as file:
	files=file.readlines()
# print(files)
# print(len(files))
print('\n')
# print(files[0])
files_list=[]
for line in files:
	line=line.strip('\n')
	line=json.loads(line)
	files_list.append(line)
# print(dict)
print(len(files_list))
# for element in dict:
	# print(element)
	# print('\n')



2248


In [10]:
with open(data_label_path,'r') as file:
	labels=file.readlines()
print(labels)
label_list=[]
for label in labels:
  label.strip('\n')
  label=json.loads(label)
  label_list.append(label)
print(label_list)

['{"indoml_id": "surprise|11109", "intent": "accept reservations"}\n', '{"indoml_id": "surprise|11051", "intent": "accept reservations"}\n', '{"indoml_id": "surprise|11021", "intent": "accept reservations"}\n', '{"indoml_id": "surprise|11047", "intent": "accept reservations"}\n', '{"indoml_id": "surprise|11004", "intent": "accept reservations"}\n', '{"indoml_id": "surprise|11097", "intent": "accept reservations"}\n', '{"indoml_id": "surprise|10998", "intent": "accept reservations"}\n', '{"indoml_id": "surprise|11050", "intent": "accept reservations"}\n', '{"indoml_id": "surprise|11035", "intent": "accept reservations"}\n', '{"indoml_id": "surprise|11030", "intent": "accept reservations"}\n', '{"indoml_id": "surprise|11127", "intent": "accept reservations"}\n', '{"indoml_id": "surprise|11042", "intent": "accept reservations"}\n', '{"indoml_id": "surprise|11105", "intent": "accept reservations"}\n', '{"indoml_id": "surprise|11139", "intent": "accept reservations"}\n', '{"indoml_id": "sur

#**Data Preprocessing for the text augmentation**

In [11]:
import pandas as pd
indoml_id_file=[]
utt=[]
indoml_id_label=[]
intent=[]


files_dict={}
label_dict={}
for file in files_list:
  indoml_id_file.append(file['indoml_id'])
  utt.append(file['utt'])
  files_dict['indoml_id']=indoml_id_file
  files_dict['utt']=utt

for file in label_list:
  indoml_id_label.append(file['indoml_id'])
  intent.append(file['intent'])
  label_dict['indoml_id']=indoml_id_label
  label_dict['intent']=intent

files_pd=pd.DataFrame.from_dict(files_dict)
files_pd



,indoml_id,utt
0,surprise|11109,Can I make a reservation at Buffalo Wild Wings?
1,surprise|11051,Can I book a table for tonight at Bella Vita?
2,surprise|11021,Does the Cheesecake Factory take reservations?
3,surprise|11047,Do they take reservations at Outback Steakhous...
4,surprise|11004,does chili's take reservations?
...,...,...
2243,surprise|5290,"indeed, that's what I was told!"
2244,surprise|5254,"Oh, without a doubt, that's absolutely true"
2245,surprise|5246,"Absolutely, I couldn't agree more!"
2246,surprise|5401,It's a wonderful notion!


In [12]:
labels_pd=pd.DataFrame.from_dict(label_dict)
labels_pd

,indoml_id,intent
0,surprise|11109,accept reservations
1,surprise|11051,accept reservations
2,surprise|11021,accept reservations
3,surprise|11047,accept reservations
4,surprise|11004,accept reservations
...,...,...
2243,surprise|5290,yes
2244,surprise|5254,yes
2245,surprise|5246,yes
2246,surprise|5401,yes


In [13]:
Data_pd=pd.merge(files_pd, labels_pd, on='indoml_id', how='inner')
Data_pd

,indoml_id,utt,intent
0,surprise|11109,Can I make a reservation at Buffalo Wild Wings?,accept reservations
1,surprise|11051,Can I book a table for tonight at Bella Vita?,accept reservations
2,surprise|11021,Does the Cheesecake Factory take reservations?,accept reservations
3,surprise|11047,Do they take reservations at Outback Steakhous...,accept reservations
4,surprise|11004,does chili's take reservations?,accept reservations
...,...,...,...
2243,surprise|5290,"indeed, that's what I was told!",yes
2244,surprise|5254,"Oh, without a doubt, that's absolutely true",yes
2245,surprise|5246,"Absolutely, I couldn't agree more!",yes
2246,surprise|5401,It's a wonderful notion!,yes


In [14]:
Data_pd.drop('indoml_id', axis=1, inplace=True)
Data_pd

,utt,intent
0,Can I make a reservation at Buffalo Wild Wings?,accept reservations
1,Can I book a table for tonight at Bella Vita?,accept reservations
2,Does the Cheesecake Factory take reservations?,accept reservations
3,Do they take reservations at Outback Steakhous...,accept reservations
4,does chili's take reservations?,accept reservations
...,...,...
2243,"indeed, that's what I was told!",yes
2244,"Oh, without a doubt, that's absolutely true",yes
2245,"Absolutely, I couldn't agree more!",yes
2246,It's a wonderful notion!,yes


# **Data augmentation using the nlpaug library**
*Here we have 2 options available in the data augmentation, we can either take synonym replacement or random insertion, where in the synonym replacement it takes every sentence of the surprise dataset and we replace words in the text with synonyms while maintaining the original context*
,
*While in the random insertion we insert new words into our original sentence at random positions. We have to add words in such a way that the context or meaning of the sentence doesn’t change,*

**Here we can take both of them for augmenting our data, or one of them, Here I have taken the synonym replacement with generating 2 more sentences for every sentence, so that new dataset size becomes the 3 times of the surprise dataset**

☁ Here user can take other options also where he can also include random insertion in his training dataset and also user can decide that how many sentence he has to generate for every sentence, but I would suggest to not take more that 4 or 5 sentences

In [15]:
import os
import torch
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import TrainingArguments
from tqdm import tqdm
import nlpaug.augmenter.word as naw
import pandas as pd
import random
import re

def clean_and_format(sentence):
    # Remove extra quotes from the beginning and end
    cleaned_sentence = sentence.lstrip("'\"").rstrip("'\"")

    # Remove square brackets and double quotes
    cleaned_sentence = cleaned_sentence.strip('["]')

    # Replace "That ' s" with "That's" or "That is"
    cleaned_sentence = re.sub(r"That ' s", "That's", cleaned_sentence)
    cleaned_sentence = re.sub(r"That ' s", "That is", cleaned_sentence)

    # Replace double quotes with single quotes between generated sentences
    cleaned_sentence = cleaned_sentence.replace('"', "'")

    # Remove extra spaces
    cleaned_sentence = re.sub(r"\s+", " ", cleaned_sentence)

    return cleaned_sentence


# Initialize WordNet-based synonym augmenter
aug_synonym = naw.SynonymAug(aug_src='wordnet')

# Initialize random insertion augmenter
aug_insert = naw.RandomWordAug()

# Augmentation settings
num_augmented_samples = 4  # Number of augmented samples per original sample

# Function to perform data augmentation
def augment_text(text):
    augmented_samples = []

    # Synonym replacement augmentation
    for _ in range(num_augmented_samples // 2):
        augmented_text = aug_synonym.augment(text)
        augmented_samples.append(augmented_text)

    # # Random insertion augmentation
    # for _ in range(num_augmented_samples // 2):
    #     augmented_text = aug_insert.augment(text)
    #     augmented_samples.append(augmented_text)

    return augmented_samples

# Augment the DataFrame and create a new DataFrame with augmented examples
augmented_data = []

for index, row in Data_pd.iterrows():
    original_text = row['utt']
    augmented_texts = augment_text(original_text)

    for augmented_text in augmented_texts:
        augmented_data.append({'utt': augmented_text, 'intent': row['intent']})

# Create a new DataFrame with augmented examples
augmented_df = pd.DataFrame(augmented_data)

# Concatenate the original DataFrame with the augmented DataFrame
final_df = pd.concat([Data_pd, augmented_df], ignore_index=True)
final_df['utt'] = final_df['utt'].astype(str)  # Ensure all values are strings

# Calculate the desired number of examples per label
desired_examples_per_label = len(final_df) * num_augmented_samples // len(final_df['intent'].unique())

# Initialize a dictionary to keep track of the number of examples per label
label_counts = {}

# List to hold the final data
final_data = []

for index, row in final_df.iterrows():
    intent_label = row['intent']

    # Check if the label has reached the desired number of examples
    if intent_label not in label_counts:
        label_counts[intent_label] = 0

    # Append the row to the final data if the label count is less than the desired count
    if label_counts[intent_label] < desired_examples_per_label:
        final_data.append({'utt': row['utt'], 'intent': intent_label})
        label_counts[intent_label] += 1

# Create the final DataFrame
final_df = pd.DataFrame(final_data)

# Clean and format the 'utt' column
final_df['utt'] = final_df['utt'].apply(lambda x: clean_and_format(x))

# Display the final DataFrame
print(final_df)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


                                                    utt               intent
0       Can I make a reservation at Buffalo Wild Wings?  accept reservations
1         Can I book a table for tonight at Bella Vita?  accept reservations
2        Does the Cheesecake Factory take reservations?  accept reservations
3     Do they take reservations at Outback Steakhous...  accept reservations
4                       does chili's take reservations?  accept reservations
...                                                 ...                  ...
6739          Absolutely, 1 couldn ' t agree more than!                  yes
6740     Information technology ' s a marvelous belief!                  yes
6741                         It ' s a marvelous belief!                  yes
6742                     That's a brilliant resolution!                  yes
6743                         That's a brilliant answer!                  yes

[6744 rows x 2 columns]


In [16]:
final_df

,utt,intent
0,Can I make a reservation at Buffalo Wild Wings?,accept reservations
1,Can I book a table for tonight at Bella Vita?,accept reservations
2,Does the Cheesecake Factory take reservations?,accept reservations
3,Do they take reservations at Outback Steakhous...,accept reservations
4,does chili's take reservations?,accept reservations
...,...,...
6739,"Absolutely, 1 couldn ' t agree more than!",yes
6740,Information technology ' s a marvelous belief!,yes
6741,It ' s a marvelous belief!,yes
6742,That's a brilliant resolution!,yes


In [17]:
final_df = final_df.sort_values(by='intent', ascending=True)

In [18]:
utt=[]
for ut in final_df['utt']:
  utt.append(ut)

In [19]:
final_df.reset_index(inplace=True)
final_df['intent']
for intent in final_df['intent']:
  print(intent)

Streaming output truncated to the last 5000 lines.
do you have pets
do you have pets
do you have pets
do you have pets
do you have pets
do you have pets
do you have pets
do you have pets
do you have pets
do you have pets
do you have pets
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
exchange rate
expiration date
expiration date
expiration date
expiration date
expiration date
expiration date
expiration date
expiration date
expi

In [20]:
intent_cleaned=[]
for label in final_df['intent']:
  intent_cleaned.append(label)

In [21]:
utt[0:5], len(utt), intent_cleaned[0:5], len(intent_cleaned)

(['Can I make a reservation at Buffalo Wild Wings?',
  "'Suffice the Pizza pie Parlor accept reservations for parties of pentad operating theatre more?'",
  "'Can I appropriate a mesa at Alinea in Chicago?'",
  "'Sack I reserve a board at Alinea in Chicago?'",
  "Where can I receive reservation selective information for Applebee ' s?"],
 6744,
 ['accept reservations',
  'accept reservations',
  'accept reservations',
  'accept reservations',
  'accept reservations'],
 6744)

In [22]:
from sklearn.model_selection import train_test_split
intents_set=set(intent_cleaned)
num_classes = len(set(intent_cleaned))
print("Number of classes")
print(num_classes)

"""Splitting the dataset into train and test set"""
train_data, test_data, train_labels, test_labels = train_test_split(
    utt, intent_cleaned, test_size=0.10, random_state=123
)

Number of classes
150


In [24]:
labels_list=[]
for label in label_list:
    labels_list.append(label['intent'])
unique_labels_list=[]
for x in labels_list:
    if x not in unique_labels_list:
        unique_labels_list.append(x)
# unique_labels_list, len(unique_labels_list)

label2id={}
id2label={}
for i, label in enumerate(unique_labels_list):
    label2id[label]=i
    id2label[i]=label

In [25]:
print(list(id2label.items())[:5])
print('\n')
print(list(label2id.items())[:5])

[(0, 'accept reservations'), (1, 'account blocked'), (2, 'alarm'), (3, 'application status'), (4, 'apr')]


[('accept reservations', 0), ('account blocked', 1), ('alarm', 2), ('application status', 3), ('apr', 4)]


# **Loading the pretrained intent classification Alexa XLMRoberta model and the tokenizer**

*Note that I have tried data augmentation with various pretrained models, you can also try with all of these*

In [26]:
# Define the BERT model and tokenizer
# keep `ignore_mismatched_sizes=True` so that the classification layer is randomly initialized
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
# model_name='roberta-large'
model_name="ibm/roberta-large-vira-intents" # by the IBM on 180 labels
tokenizer=RobertaTokenizer.from_pretrained(model_name)

In [27]:
# Initialize the model
num_classes=150
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_classes, ignore_mismatched_sizes=True)

"""since model have its own id2label mapping and vice versa so, converting them from our id2label and label2id mapping as defined earlier"""
model.label2id=label2id
model.id2label=id2label


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ibm/roberta-large-vira-intents and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([181, 1024]) in the checkpoint and torch.Size([150, 1024]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([181]) in the checkpoint and torch.Size([150]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
# Tokenize the input training data
train_encodings = tokenizer(
    train_data,
    truncation=True,
    padding=True,
    max_length=64,
    return_tensors='pt'    #return type is pytorch tensor
)
#Tokenizing the input testing data
test_encodings = tokenizer(
    test_data,
    truncation=True,
    padding=True,
    max_length=64,
    return_tensors='pt'
)

In [29]:
"""Convert labels to numeric values from label2id mapping"""
train_int_labels = [label2id[label] for label in train_labels]
test_int_labels = [label2id[label] for label in test_labels]
test_int_labels[0:5]

[149, 30, 21, 106, 19]

# **making pytorch datasets**

In [ ]:
# Create PyTorch datasets
class IntentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IntentDataset(train_encodings, train_int_labels)
test_dataset = IntentDataset(test_encodings, test_int_labels)

In [ ]:
from torch.utils.data import DataLoader
batch_size = 64  # Adjust the batch size as needed
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True  # You can shuffle your data for randomness during training
)
eval_dataloader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False # You can shuffle your data for randomness during training
)


# **initializing training arguments and training the model**

In [ ]:
# Training arguments
# previous_saved_dir='./intent_classification__'
saved_dir='/home/naive123/nlp/Sumit/massive_git/massive/phase_2_jupter_files/trained_model_3_again_fine_tuning_checkpoints/'

training_args = TrainingArguments(
    output_dir=saved_dir,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    seed=42,
    # weight-decay=0.01
    # gradient_accumulation_steps=40,
    evaluation_strategy="steps", # use 'epoch' for evaluating every epoch
    logging_steps=10,
    eval_steps=10,
    save_total_limit=5,
    save_steps=60,
    learning_rate=2e-5,
    # warmup_steps=400,
    # weight_decay=0.10,
    # adam_epsilon=1e-7,
    # warmup_steps=400,
    num_train_epochs=40,
    logging_dir='./logs',
)

In [ ]:
import os
import torch
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import TrainingArguments
from sklearn.model_selection import KFold
from tqdm import tqdm

# Define your model, tokenizer, and other necessary components here
# Make sure your model is moved to the GPU if available

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the learning rate as a float (e.g., 7e-6)
learning_rate = 7e-5

# Define the number of training epochs as an integer (e.g., 40)
num_train_epochs = 40

# Create the optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Define the number of training steps (you may need to adjust this based on your dataset)
total_train_steps = len(train_dataloader) * num_train_epochs

# Define the number of warmup steps (e.g., 10% of the total training steps)
num_warmup_steps = int(0.1 * total_train_steps)

# Create a learning rate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=total_train_steps,
)

# Define the output directory for saving the model
# saved_dir='/home/naive123/nlp/Sumit/massive_git/massive/phase_2_jupter_files/trained_model_3_again_fine_tuning_checkpoints/'
# Directory where all weights will get saved
saved_dir='/content/drive/MyDrive/intent_classification_It_bombay/trained_model_1_direct_roberta_large_text_aug_synonym_6744_checkpoints/'
# Training loop with tqdm progress bar
for epoch in range(num_train_epochs):
    model.train()
    train_loss = 0.0

    progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch}")

    for step, batch in progress_bar:
        # Move data to the GPU
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        # Update the learning rate
        lr_scheduler.step()

        # Accumulate the total loss
        train_loss += loss.item()

        # Log loss and other metrics if needed
        if step % training_args.logging_steps == 0:
            avg_loss = train_loss / (step + 1)
            progress_bar.set_postfix(loss=avg_loss)

    # Save the model checkpoint at the end of each epoch
    checkpoint_dir = os.path.join(saved_dir, f"epoch_{epoch}")
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Save model weights
    model.save_pretrained(checkpoint_dir)

    # Save optimizer state
    # torch.save(optimizer.state_dict(), os.path.join(checkpoint_dir, "optimizer.pt"))

    # # Save learning rate scheduler state
    # torch.save(lr_scheduler.state_dict(), os.path.join(checkpoint_dir, "scheduler.pt"))

    # Save RNG state (if needed)
    # torch.save(torch.get_rng_state(), os.path.join(checkpoint_dir, "rng_state.pth"))

    # Save config.json (if needed)
    model.config.save_pretrained(checkpoint_dir)

    # Save tokenizer (if needed)
    tokenizer.save_pretrained(checkpoint_dir)

    # Evaluation loop with tqdm progress bar
    model.eval()
    eval_loss = 0.0

    progress_bar_eval = tqdm(enumerate(eval_dataloader), total=len(eval_dataloader), desc=f"Evaluation")

    for step, batch in progress_bar_eval:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
            loss = outputs.loss
            eval_loss += loss.item()

    avg_eval_loss = eval_loss / len(eval_dataloader)
    print(f"Epoch {epoch}: Evaluation Loss: {avg_eval_loss}")
    print(f'learning rate is : {learning_rate}')

    # if (epoch + 1) % 3 == 0:
    #     learning_rate *= 0.75
    #     for param_group in optimizer.param_groups:
    #         param_group['lr'] = learning_rate


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Evaluation: 100%|██████████| 11/11 [00:05<00:00,  2.03it/s]


Epoch 0: Evaluation Loss: 4.808306043798273
learning rate is : 7e-05


Evaluation: 100%|██████████| 11/11 [00:05<00:00,  2.05it/s]


Epoch 1: Evaluation Loss: 1.4738577712665906
learning rate is : 7e-05


Evaluation: 100%|██████████| 11/11 [00:05<00:00,  2.04it/s]


...

Epoch 10: Evaluation Loss: 0.07853100465779955
learning rate is : 7e-05


Evaluation: 100%|██████████| 11/11 [00:05<00:00,  2.04it/s]


Epoch 11: Evaluation Loss: 0.09082011802291329
learning rate is : 7e-05


Epoch 12:   9%|▉         | 9/95 [00:20<03:13,  2.25s/it, loss=0.0138]


KeyboardInterrupt: ignored

# **prediction on the test set(whose labels is available)**

In [ ]:
import numpy as np
import torch
# taking the best model from above training used for getting the predictions on the validation dataset
model_name="/content/drive/MyDrive/intent_classification_It_bombay/trained_model_1_direct_roberta_large_text_aug_synonym_6744_checkpoints/epoch_9"
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_classes, ignore_mismatched_sizes=True)

# Put your test data into a DataLoader or batch format if it's not already
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

model.to(device)
# Set the model to evaluation mode
model.eval()

predicted_labels = []
raw_predictions = []

with torch.no_grad():
    for batch in test_dataloader:
        # Move batch to the GPU if available
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        logits = outputs.logits

        # Get predicted labels (argmax)
        batch_predictions = np.argmax(logits.cpu().numpy(), axis=1)

        # Append batch predictions to the list
        predicted_labels.extend(batch_predictions)

        # Append raw logits if needed
        raw_predictions.extend(logits.cpu().numpy())

# Convert the results to numpy arrays
predicted_labels = np.array(predicted_labels)
raw_predictions = np.array(raw_predictions)

# Now, you have predicted labels and raw logits.


In [ ]:
predicted_labels, raw_predictions

(array([ 95,  59,  71,  91, 137,  99, 135,  20,  18, 118,  24,  78, 134,
         29,  57,  94,  93, 118,  66,  50,  86,  19, 128, 109,  83, 144,
        107, 124, 103,  44,  67,  70,  95,  35,  52, 136,  64, 114,  29,
         63,  76,  45, 148, 126,  39,   8,  97,  67,  12, 147, 139,  88,
         78,  22, 126, 145,  94, 129, 140,  60,  97,  25,  78, 140, 130,
         11, 136,  29, 104,  97, 128,  92,  31,  44,  59,  59, 121,  43,
         44,  90, 108,  35,   7,  12, 127, 101, 117, 129,  33, 141, 107,
        111,  10, 143,  98,  63, 131,  94,  72,  33,  41,  68, 100,  90,
         82,  44,  58, 126, 124,  88,  95, 121,  22,   7,  26, 116,  87,
        134,  47,  89,  30, 136,  31,  35,  76,  87, 138,  82, 102,  80,
          6,  74,  36,  85, 117,  92,  26,  64,  37,  88,  34,  85,   5,
         61,   2, 129,  71, 111,  49,  15, 100, 143,   0,  75,  20,  37,
         76,  80,  45, 137, 149,  50,  70,  14,  42,  46, 138, 141,  16,
         29, 112,  79,  27, 117,  16, 132,  58, 109

In [ ]:
# len(raw_predictions.predictions), raw_predictions.predictions

In [ ]:
# raw_predictions = trainer.predict(test_dataset)

In [ ]:

# # Predict on the test set
# raw_predictions = trainer.predict(test_dataset)
# predicted_labels = np.argmax(raw_predictions.predictions, axis=1)

# # Convert labels back to original intents
# # predicted_intents = [list(label_map.keys())[list(label_map.values()).index(label)] for label in predicted_labels]
# predicted_intents=[id2label[label] for label in predicted_labels]

# # Print one sample prediction
# sample_idx = random.randint(0, len(test_data) - 1)
# sample_text = test_data[sample_idx]
# sample_intent = predicted_intents[sample_idx]

# print(f"Sample Text: {sample_text}")
# print(f"Predicted Intent: {sample_intent}")
import random
import numpy as np

# Put your test data into a DataLoader or batch format if it's not already
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Set the model to evaluation mode
model.eval()

predicted_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        # Move batch to the GPU if available
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        logits = outputs.logits

        # Get predicted labels (argmax)
        batch_predictions = np.argmax(logits.cpu().numpy(), axis=1)

        # Append batch predictions to the list
        predicted_labels.extend(batch_predictions)

# Convert the results to numpy arrays
predicted_labels = np.array(predicted_labels)

# Convert labels back to original intents
predicted_intents = [id2label[label] for label in predicted_labels]

# Print one sample prediction
sample_idx = random.randint(0, len(test_data) - 1)
sample_text = test_data[sample_idx]
sample_intent = predicted_intents[sample_idx]

print(f"Sample Text: {sample_text}")
print(f"Predicted Intent: {sample_intent}")


Sample Text: I want to switch to a different auto insurance provider.
Predicted Intent: insurance change


# **metrics(accuracy criterias)**

In [ ]:
# Calculate accuracy, precision, recall, and F1-score
report = classification_report(test_int_labels, predicted_labels, target_names=list(label2id.keys()), output_dict=True)
print(report)
print('\n')
accuracy = report['accuracy']
precision = report['macro avg']['precision']
recall = report['macro avg']['recall']
f1_score = report['macro avg']['f1-score']

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1_score:.4f}")

{'distance': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 4}, 'current location': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 4}, 'user name': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 5}, 'bill due': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 4}, 'text': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 4}, 'what are your hobbies': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 4}, 'how busy': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 4}, 'international fees': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 4}, 'what song': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 5}, 'account blocked': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 5}, 'cook time': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 5}, 'damaged card': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 5}, 'ingredient substituti

In [ ]:
report

{'distance': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 4},
 'current location': {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0,
  'support': 4},
 'user name': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 5},
 'bill due': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 4},
 'text': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 4},
 'what are your hobbies': {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0,
  'support': 4},
 'how busy': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 4},
 'international fees': {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0,
  'support': 4},
 'what song': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 5},
 'account blocked': {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0,
  'support': 5},
 'cook time': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 5},
 'damaged card': {'precision': 1.0,
  'recall': 1.0,
  'f1-score': 1.0

In [ ]:
del report['accuracy']

In [ ]:
import pandas as pd

# Initialize an empty list to store the data as dictionaries
data_list = []

# Iterate through the data_dict and convert each entry into a dictionary
for label, metrics in report.items():
    entry = {'label': label}
    print(entry)
    entry.update(metrics)

    data_list.append(entry)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

# Print the resulting DataFrame
print(df)


{'label': 'distance'}
{'label': 'current location'}
{'label': 'user name'}
{'label': 'bill due'}
{'label': 'text'}
{'label': 'what are your hobbies'}
{'label': 'how busy'}
{'label': 'international fees'}
{'label': 'what song'}
{'label': 'account blocked'}
{'label': 'cook time'}
{'label': 'damaged card'}
{'label': 'ingredient substitution'}
{'label': 'pay bill'}
{'label': 'uber'}
{'label': 'spelling'}
{'label': 'schedule maintenance'}
{'label': 'roll dice'}
{'label': 'change accent'}
{'label': 'order checks'}
{'label': 'report fraud'}
{'label': 'new card'}
{'label': 'flight status'}
{'label': 'yes'}
{'label': 'replacement card duration'}
{'label': 'next song'}
{'label': 'min payment'}
{'label': 'vaccines'}
{'label': 'directions'}
{'label': 'measurement conversion'}
{'label': 'weather'}
{'label': 'calendar'}
{'label': 'translate'}
{'label': 'income'}
{'label': 'date'}
{'label': 'balance'}
{'label': 'report lost card'}
{'label': 'international visa'}
{'label': 'carry on'}
{'label': 'make 

In [ ]:
df.to_csv('label_list.csv')